### Импорт библиотек для работы с бд и дф

In [1]:
import psycopg2
import pandas as pd
import sqlalchemy

### Кредиты для коннекта к бд

In [8]:
conn = psycopg2.connect(host="127.0.0.1", port="5432", database="Adventureworks", 
                            user="postgres", password="postgres")

In [5]:
query = f'''

 select 
        storeid,
        salesorderid,
        subtotal,
        orderdate
        from sales.customer l
        join sales.salesorderheader r
        on l.customerid = r.customerid
        where storeid in  (292, 300,658)
  
'''
df = pd.read_sql_query(query, conn)
df.head(20)

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,storeid,salesorderid,subtotal,orderdate
0,292,44132,4049.9880,2011-08-01
1,292,45579,4079.9880,2012-01-29
2,292,46389,1104.9968,2012-04-30
3,658,47388,202.3320,2012-07-31
4,292,47454,27429.5294,2012-07-31
5,658,48329,1070.0565,2012-10-30
6,292,48395,32562.6538,2012-10-30
7,292,49495,24232.7654,2013-01-28
8,658,50679,386.2702,2013-04-30
9,292,50756,37643.0609,2013-04-30


### проблема производительности, поскольку PostgreSQL должен сканировать таблицу отдельно для каждого запроса.
### Чтобы сделать его более эффективным, PostgreSQL предоставляет предложение GROUPING sets
### НАБОРЫ ГРУППИРОВКИ позволяют определять несколько наборов группирования в одном запросе.

In [19]:
query = f'''

 select 
        storeid,
        count(salesorderid),
        sum(subtotal),
        orderdate
        from sales.customer l
        join sales.salesorderheader r
        on l.customerid = r.customerid
        where storeid in  (292, 300,658)
        group by 
        grouping sets (storeid, (storeid, orderdate))
  
'''
df = pd.read_sql_query(query, conn)
df.head(40)

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,storeid,count,sum,orderdate
0,292,1,4049.9880,2011-08-01
1,292,1,4079.9880,2012-01-29
2,292,1,1104.9968,2012-04-30
3,292,1,27429.5294,2012-07-31
4,292,1,32562.6538,2012-10-30
5,292,1,24232.7654,2013-01-28
6,292,1,37643.0609,2013-04-30
7,292,7,131102.9823,NaT
8,300,1,57771.7641,2013-07-31
9,300,1,49053.4638,2013-10-30


### ROLLUP - это подитог для предложения GROUP BY, который предлагает сокращенное обозначение для определения нескольких наборов группировки.  


In [31]:
query = f'''

 select 
        storeid,
        count(salesorderid),
        sum(subtotal),
        date_trunc('year', orderdate)
        from sales.customer l
        join sales.salesorderheader r
        on l.customerid = r.customerid
        where storeid in  (292, 300,658)
        group by 
        grouping sets (storeid, (storeid,  date_trunc('year', orderdate)), rollup (storeid))
  
'''
df = pd.read_sql_query(query, conn)
df.head(30)

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,storeid,count,sum,date_trunc
0,292.0,1,4049.9880,2011-01-01
1,292.0,4,65177.1680,2012-01-01
2,292.0,2,61875.8263,2013-01-01
3,292.0,7,131102.9823,NaT
4,292.0,7,131102.9823,NaT
5,300.0,2,106825.2279,2013-01-01
6,300.0,2,114344.5566,2014-01-01
7,300.0,4,221169.7845,NaT
8,300.0,4,221169.7845,NaT
9,658.0,2,1272.3885,2012-01-01


In [30]:
query = f'''

 select 
        storeid,
        count(salesorderid),
        sum(subtotal)
        from sales.customer l
        join sales.salesorderheader r
        on l.customerid = r.customerid
        where storeid in  (292, 300,658)
        group by rollup (storeid)
  
'''
df = pd.read_sql_query(query, conn)
df.head(30)

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,storeid,count,sum
0,NaN,18,359456.3455
1,300.0,4,221169.7845
2,658.0,7,7183.5787
3,292.0,7,131102.9823


### CUBE используется в основном для целей отчетности. С помощью CUBE пользователь может расширить функциональность предложений GROUP BY, 
### вычислив общие значения (для каждого из наборов). Перебор данных гораздо больше.
### CUBE(X,Y,Z) создает grouping sets для всех возможных значений: 

In [32]:
query = f'''

 select 
        storeid,
        count(salesorderid),
        sum(subtotal),
        date_trunc('year', orderdate)
        from sales.customer l
        join sales.salesorderheader r
        on l.customerid = r.customerid
        where storeid in  (292, 300,658)
        group by cube (storeid, date_trunc('year', orderdate))
  
'''
df = pd.read_sql_query(query, conn)
df.head(30)

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,storeid,count,sum,date_trunc
0,NaN,18,359456.3455,NaT
1,300.0,2,114344.5566,2014-01-01
2,300.0,2,106825.2279,2013-01-01
3,658.0,3,5049.5422,2013-01-01
4,292.0,2,61875.8263,2013-01-01
5,658.0,2,861.6480,2014-01-01
6,292.0,1,4049.9880,2011-01-01
7,658.0,2,1272.3885,2012-01-01
8,292.0,4,65177.1680,2012-01-01
9,300.0,4,221169.7845,NaT


In [26]:
conn.rollback()
conn.close()